# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_fr import lexicon_fr
from utils.io import boreholes_from_files, gdf_viewer
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [3]:
df = pd.read_csv(save_dir+'Pilote_test.csv', sep=',')

In [4]:
gdf_viewer(df)

Rows : 87, columns : 9


interactive(children=(IntSlider(value=10, description='rows', max=87, min=10, readout=False), IntSlider(value=…

## Data Checking

In [5]:
df.drop(index=df.query('X!=X').index, inplace=True)
df.drop(index=df.query('Description!=Description').index, inplace=True)

In [6]:
df.query('ID=="201"')

,ID,X,Y,Z,Litho_top,Description,Litho_base,Diam_for,Long_for


In [7]:
df['Ep_litho']=df['Litho_base']-df['Litho_top']
df

,ID,X,Y,Z,Litho_top,Description,Litho_base,Diam_for,Long_for,Ep_litho
0,F201,152890.245758,122596.474260,101.926886,0.6,Remblais avec briques de construction couleur ...,1.2,75.0,2.4,0.6
1,F201,152890.245758,122596.474260,101.926886,1.2,Briquaille de déchets de construction et galet...,2.4,75.0,2.4,1.2
2,F205,152887.693175,122594.619977,101.804167,0.0,Couche de boue en surface liée à l'interventio...,0.2,75.0,4.8,0.2
3,F205,152887.693175,122594.619977,101.804167,0.2,Limon brun,0.4,75.0,4.8,0.2
4,F205,152887.693175,122594.619977,101.804167,0.4,Remblais gris gravillonneux avec présence de d...,1.2,75.0,4.8,0.8
5,F205,152887.693175,122594.619977,101.804167,1.2,Présence d'eau avec remblais pollués (odeur fo...,2.4,75.0,4.8,1.2
6,F205,152887.693175,122594.619977,101.804167,2.4,Remblais grossiers saturés avec présence de pr...,3.2,75.0,4.8,0.8
7,F205,152887.693175,122594.619977,101.804167,3.2,Alluvions verdâtres argilo-sableux (odeur dans...,3.6,75.0,4.8,0.4
8,F205,152887.693175,122594.619977,101.804167,3.6,Retombées des strates supérieures - Limon verd...,4.8,75.0,4.8,1.2
9,F208,152885.295648,122592.985739,101.847697,0.0,Remblais de couleur brun avec déchets de const...,1.2,75.0,4.8,1.2


## Create Boreholes from dataframes or files

In [8]:
bhs_df=[df] # list of dataframes

In [10]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Description',
                                                lexicon=lexicon_fr,
                                                diam_field='Diam_for', 
                                                litho_top_field='Litho_top',
                                                litho_base_field='Litho_base',
                                                thick_field='Ep_litho')


Dataframe 0 processing...
|__ID:'F201'
Summary : [Striplog(2 Intervals, start=0.6, stop=2.4)]
{0: {'description': 'Remblais avec briques de construction couleur orange, gris, verdâtre  ', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7ff1a5533100>, 'base': <core.orm.PositionOrm object at 0x7ff1a54eb640>}, 1: {'description': "Briquaille de déchets de construction et galets roulés - présence d'eau - légère odeur de naphtalène - Bloqué à 2,4 m sur caillasse (et béton ?)", 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7ff1a53cf3d0>, 'base': <core.orm.PositionOrm object at 0x7ff1a5313370>}}

|__ID:'F205'
Error : No lithology matching with 'Couche de boue en surface liée à l'intervention de Géosonda' in given lexicon
Summary : [Striplog(7 Intervals, start=0.0, stop=4.8)]
{2: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7ff1a5313ac0>, 'base': <core.orm.PositionOrm object at 0x7ff1a5313a00>}, 3: {'description': 

In [26]:
litho_list = []
color_list = []
for i, c in components.items():
    if not hasattr(c, 'lithology'):
        c.lithology = 'sable'
        c.colour = 'rouge'
    elif c.lithology == 'sand':
        c.lithology = 'sable'
        c.colour = 'blanc'
    if c.lithology not in litho_list:
        litho_list.append(c.lithology)
    if hasattr(c, 'colour') and c.colour not in color_list:
        color_list.append(c.colour)

In [27]:
litho_list, color_list

(['sable', 'remblais', 'limon', 'limons', 'schistes'],
 ['rouge',
  'gris verdâtre',
  'verdâtre',
  'gris',
  'blanc',
  'sableuse',
  'noirs',
  'noirs gris',
  'noire',
  'noir verdâtre',
  'foncé',
  'gris foncé',
  'jaune',
  'noir'])

In [13]:
memoris_components = [Component({'lithology': 'remblais'}),
              Component({'lithology': 'limon'}),
              Component({'lithology': 'schistes'}),
              Component({'lithology': 'sable'}),
              Component({'lithology': 'anthropique'})
              ]
list_of_decors=[]
hatches = ['=', 'x', '.', 's', '=']
colours = ['#888888', '#882222', '#AAAAAA', '#CC22CC', '#CC2222']

for i in range(len(memoris_components)):
    if hasattr(memoris_components[i], 'colour'):
        c = memoris_components[i].colour
    else:
        c=colours[i]
    
    if hasattr(memoris_components[i], 'hatch'):
        h = memoris_components[i].hatch
    else:
        h=hatches[i]
        
    d = {'color': c,
         'hatch': h,
         'component': memoris_components[i],
         'width': 3}
    decor = Decor(d)
    list_of_decors.append(decor)
    
memoris_legend = Legend(list_of_decors)

In [14]:
boreholes

[<core.orm.BoreholeOrm>(Name=F201, Length=1.7999999999999998, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=F205, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=F208, Length=4.8, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=F212, Length=4.8, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=F207, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=F214, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=F217, Length=4.2, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=F221, Length=1.4, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=F223, Length=1.3, Diameter=75.0, Intervals=1),
 <core.orm.BoreholeOrm>(Name=F225, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=F224, Length=2.4, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=F219, Length=1.5, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=F220, Length=0.5, Diameter=75.0, Intervals=1

In [15]:
components

{0: Component({'material': 'béton', 'grainsize': 'galets', 'Pollutant': 'naphtalène', 'lithology': 'sable', 'colour': 'rouge'}),
 1: Component({'lithology': 'remblais', 'colour': 'gris verdâtre'}),
 2: Component({'lithology': 'remblais', 'Pollutant': 'naphtalène'}),
 3: Component({'lithology': 'limon', 'colour': 'verdâtre', 'grainsize': 'grossiers'}),
 4: Component({'lithology': 'remblais', 'grainsize': 'grossiers', 'Pollutant': 'naphtalène'}),
 5: Component({'lithology': 'remblais', 'colour': 'gris'}),
 6: Component({'modifier': 'sableux', 'colour': 'rouge', 'lithology': 'sable'}),
 7: Component({'lithology': 'sable', 'colour': 'blanc'}),
 8: Component({'lithology': 'limon'}),
 9: Component({'lithology': 'remblais', 'modifier': 'sableuse', 'colour': 'sableuse'}),
 10: Component({'grainsize': 'grossiers', 'lithology': 'sable', 'colour': 'rouge'}),
 11: Component({'lithology': 'limons', 'colour': 'verdâtre'}),
 12: Component({'lithology': 'remblais', 'grainsize': 'grossiers'}),
 13: Com

In [16]:
len(boreholes), len(components)

(15, 45)

## Create a project and save data in a database

In [17]:
remove('tmp_files/tfe_orm_db.db')

In [18]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [19]:
Base.metadata.create_all(engine)

2021-06-16 13:10:45,211 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-16 13:10:45,212 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-06-16 13:10:45,213 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 13:10:45,215 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-06-16 13:10:45,216 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 13:10:45,217 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-06-16 13:10:45,217 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 13:10:45,219 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-06-16 13:10:45,219 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 13:10:45,221 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-06-16 13:10:45,222 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 13:10:45,223 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-06-16 13:10:45,225 INFO sqlalchemy.engine.Engine 

In [20]:
Session = sessionmaker(bind=engine)
session = Session()

In [21]:
p = Project(session, legend=memoris_legend, lexicon=lexicon_fr)
p.add_components(components)

2021-06-16 13:11:00,835 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-16 13:11:00,840 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-16 13:11:00,842 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ()
2021-06-16 13:11:00,857 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-06-16 13:11:00,858 INFO sqlalchemy.engine.Engine [generated in 0.00073s] ((0, 'Béton, galets, naphtalène, sable, rouge'), (1, 'Remblais, gris verdâtre'), (2, 'Remblais, naphtalène'), (3, 'Limon, verdâtre, grossiers'), (4, 'Remblais, grossiers, naphtalène'), (5, 'Remblais, gris'), (6, 'Sableux, rouge, sable'), (7, 'Sable, blanc')  ... displaying 10 of 45 total bound parameter sets ...  (43, 'Remblais, saturé'), (44, 'Limon, argileux, verdâtre'))
2021-06-16 13:11:00,861 INFO sqlalchemy.engine.Engine COMMIT
2021-06-16 13:11:00,86

Add boreholes into the project

In [22]:
for bh in boreholes:
    p.add_borehole(bh)

2021-06-16 13:11:02,280 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-16 13:11:02,282 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ('F201', 1.7999999999999998, 75.0)
2021-06-16 13:11:02,285 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-16 13:11:02,286 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ((0, 0.6, 0.6, 0.6, 152890.245758, 122596.47426), (1, 1.2, 1.2, 1.2, 152890.245758, 122596.47426), (2, 1.2, 1.2, 1.2, 152890.245758, 122596.47426), (3, 2.4, 2.4, 2.4, 152890.245758, 122596.47426))
2021-06-16 13:11:02,290 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-06-16 13:11:02,292 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ((0, 'F201', 0, 'Remblais avec briques de construction couleur orange, gris, verdâtre  ', 0, 1), (1, 'F201', 1, "

LegendError: Colour not recognized: gris verdâtre

In [ ]:
components
#boreholes[0].intervals[0].components

In [ ]:
p.add_link_between_components_and_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

In [ ]:
p.boreholes

In [ ]:
p.refresh(verbose=True)

In [ ]:
session.close()

# 3D Display

## Display one object

In [ ]:
bh = p.boreholes_3d
#print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

In [ ]:
p.boreholes_3d[0].plot2d()

In [ ]:
p.boreholes_3d[0].plot3d()

In [ ]:
p.boreholes_3d[0].plot3d(x3d=True)

### Display project boreholes

In [ ]:
p.plot3d(labels_size=15, bg_color=["royalblue", "aliceblue"], window_size=(1300, 1000))

### Boreholes in 3D using X3D for interaction within the notebook